# The `pandas` Groupby

**Author:** Marco "Milo" Hemken

## A quick reference guide

This is intended as a quick reference for using the `groupby` method in `pandas`. As of 5 Nov 2017 it is under construction. The examples in this notebook have been shamelessly stolen from Wes McKinney's book, [Python for Data Analysis, Second Edition](http://shop.oreilly.com/product/0636920050896.do). Go there to learn more.

In [1]:
# The maths, graphs, stats and style libs

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy import stats
import matplotlib.style as mplstyle
%matplotlib inline
mplstyle.use('fivethirtyeight')

### First example

Here we create a DataFrame object. Think of it like a spreadsheet but outside of excel. It is just a group of columns and rows with numbers. Once we've created it, we tell Python to display the data.

In [2]:
# Create a set of data from random numbers

df = pd.DataFrame({
    'key1': 'a a b b a'.split() * 2,
    'key2': 'one two one two one'.split() * 2,
    'data1': np.random.chisquare(100, 10),  # Creates random data
    'data2': np.random.chisquare(100, 10)  # Creates random data
})

df  # Show the data frame

,data1,data2,key1,key2
0,86.599396,80.962552,a,one
1,83.956394,93.040950,a,two
2,87.777299,88.208691,b,one
3,87.695403,107.368675,b,two
4,113.230065,108.345152,a,one
5,92.614460,108.706743,a,one
6,99.505888,100.214085,a,two
7,77.311499,96.963462,b,one
8,106.239788,80.799738,b,two
9,93.748245,101.906838,a,one


Now we want to see the average of all the `a`'s and `b`'s. So we create a group object. Once we have a group object we can ask it questions like "what's the average?", "what's the median?" etc.

In [3]:
g = df.groupby('key1')

g  # Show the grouped object

In [4]:
# What is the average by group?

g.mean()

,data1,data2
key1,,
a,94.942408,98.862720
b,89.755997,93.335141


In [5]:
# What is the median by group?

g.median()

,data1,data2
key1,,
a,93.181352,101.060461
b,87.736351,92.586076


In [6]:
# What is the standard deviation by group?

g.std()

,data1,data2
key1,,
a,10.511018,10.509264
b,12.038026,11.453116


## Multiple layers of grouping?

In [7]:
m = df['data1'].groupby([df['key1'], df['key2']])

In [8]:
m.median()

key1  key2
a     one     93.181352
      two     91.731141
b     one     82.544399
      two     96.967596
Name: data1, dtype: float64

In this summary we have the word 'one' appearing twice. Same with the word 'two'. That is visually inefficient because we have this stack of ones and twos there and we can't quickly compare side by side...

## And check this out...

In [9]:
m.mean().unstack()

key2,one,two
key1,,
a,96.548041,91.731141
b,82.544399,96.967596


Natually this would only work nicely with two dimensions. I wonder what happens with three.

In [10]:
df2 = pd.DataFrame({
    'key1': 'a a b b a'.split(),
    'key2': 'one two one two one'.split(),
    'key3': 'fee fi foe foe fum'.split(),
    'data1': np.random.chisquare(100, 5),
    'data2': np.random.chisquare(100, 5),
    'data3': np.random.chisquare(100, 5)
})

In [11]:
df2

,data1,data2,data3,key1,key2,key3
0,104.882983,78.221239,123.864369,a,one,fee
1,101.740491,106.499655,76.072898,a,two,fi
2,119.480875,89.779969,109.032481,b,one,foe
3,117.710040,118.952888,100.460124,b,two,foe
4,91.380973,85.670608,77.560391,a,one,fum


In [12]:
t = df2['data1'].groupby([df2['key1'], df2['key2'], df2['key3']])

In [13]:
t.mean()

key1  key2  key3
a     one   fee     104.882983
            fum      91.380973
      two   fi      101.740491
b     one   foe     119.480875
      two   foe     117.710040
Name: data1, dtype: float64

In [14]:
t.mean().unstack()

key3              fee          fi         foe        fum
key1 key2                                               
a    one   104.882983         NaN         NaN  91.380973
     two          NaN  101.740491         NaN        NaN
b    one          NaN         NaN  119.480875        NaN
     two          NaN         NaN  117.710040        NaN

Well I'll be damned it still behaves nicely. But still doesn't work as well as the two dimensional example.

## Group keys

They don't have to be part of the dataframe. They just have to be arrays of the right length.

In [15]:
df = pd.DataFrame({
    'key1': 'a a b b a'.split(),
    'key2': 'one two one two one'.split(),
    'data1': np.random.chisquare(100, 5),  # Creates random data
    'data2': np.random.chisquare(100, 5)  # Creates random data
})

In [16]:
states = np.array('Ohio California California Ohio Ohio'.split())

In [17]:
years = np.array([2005, 2005, 2006, 2005, 2006])

In [18]:
df['data1'].groupby([states, years]).mean()

California  2005     71.297520
            2006    102.028623
Ohio        2005     95.999168
            2006     80.292342
Name: data1, dtype: float64

Wow. I'm amazed. This is too easy.

In [19]:
# But if they are part of the dataframe, there is a shortcut

df.groupby('key1').mean()

,data1,data2
key1,,
a,82.638966,90.912456
b,98.849961,109.348451


In [20]:
df.groupby(['key1', 'key2']).mean()

data1       data2
key1 key2                        
a    one    88.309689   90.032136
     two    71.297520   92.673094
b    one   102.028623  101.530417
     two    95.671300  117.166485

In [21]:
# And a useful aggregator is 

df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

## Iterating over groups

In [22]:
# With a single group key

for name, group in df.groupby('key1'):
    print(name)
    print(group.std())

a
data1    12.678689
data2     6.904878
dtype: float64
b
data1     4.495306
data2    11.056370
dtype: float64


In [23]:
# With multiple group keys, the first element is always a tuple

for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group.mean(), '\n')

('a', 'one')
data1    88.309689
data2    90.032136
dtype: float64 

('a', 'two')
data1    71.297520
data2    92.673094
dtype: float64 

('b', 'one')
data1    102.028623
data2    101.530417
dtype: float64 

('b', 'two')
data1     95.671300
data2    117.166485
dtype: float64 



### Nice recipe here

In [24]:
pieces = dict(list(df.groupby('key1')))

In [25]:
pieces['b']

,data1,data2,key1,key2
2,102.028623,101.530417,b,one
3,95.671300,117.166485,b,two


In [26]:
df

,data1,data2,key1,key2
0,96.327036,96.766559,a,one
1,71.297520,92.673094,a,two
2,102.028623,101.530417,b,one
3,95.671300,117.166485,b,two
4,80.292342,83.297714,a,one


## Axis 1 grouping

In [27]:
df.dtypes

data1    float64
data2    float64
key1      object
key2      object
dtype: object

In [28]:
g = df.groupby(df.dtypes, axis=1)

In [29]:
for dtype, group in g:
    print(dtype)
    print(group, '\n')

float64
        data1       data2
0   96.327036   96.766559
1   71.297520   92.673094
2  102.028623  101.530417
3   95.671300  117.166485
4   80.292342   83.297714 

object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one 



## Selecting a column or subset of columns

In [30]:
# This,

a = df.groupby('key1')['data1']
a

In [31]:
# is the same as this

b = df['data1'].groupby(df['key1'])
b

In [32]:
# check it

print(a.mean(), '\n')
print(b.mean())

key1
a    82.638966
b    98.849961
Name: data1, dtype: float64 

key1
a    82.638966
b    98.849961
Name: data1, dtype: float64


In [33]:
# Getting fancy with it

df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2            
a    one    90.032136
     two    92.673094
b    one   101.530417
     two   117.166485

Objects returned are pd.DataFrames unless a single column is used. In that case it is a Series.

In [34]:
s_grouped = df.groupby(['key1', 'key2'])['data2']

s_grouped

In [35]:
s_grouped.mean()

key1  key2
a     one      90.032136
      two      92.673094
b     one     101.530417
      two     117.166485
Name: data2, dtype: float64

## Grouping with Dicts and Series

You can create a mapping of columns. Maybe a few columns are similare and they should be aggregated together but you need something to aggregate them by. So you can use a dictionary for that. And because this is a way of grouping columns, it makes sense that we use `axis=1`.

In [36]:
people = pd.DataFrame(np.random.randn(5, 5),
                     columns='a b c d e'.split(),
                     index='Joe Steve Wes Jim Travis'.split())
people

,a,b,c,d,e
Joe,0.907492,-1.215566,0.731149,-0.783882,0.713356
Steve,-0.688385,0.197563,0.994769,1.182886,-0.854856
Wes,-1.346094,-0.644787,-0.556969,1.246634,-0.376605
Jim,-0.493079,-0.458175,0.282342,0.089503,1.515608
Travis,0.144587,0.014133,-1.065756,-0.178602,0.347893


In [37]:
people.iloc[2:3, [1, 2]] = np.nan

people

,a,b,c,d,e
Joe,0.907492,-1.215566,0.731149,-0.783882,0.713356
Steve,-0.688385,0.197563,0.994769,1.182886,-0.854856
Wes,-1.346094,NaN,NaN,1.246634,-0.376605
Jim,-0.493079,-0.458175,0.282342,0.089503,1.515608
Travis,0.144587,0.014133,-1.065756,-0.178602,0.347893


In [38]:
mapping = {
    'a': 'red',
    'b': 'red',
    'c': 'blue',
    'd': 'blue',
    'e': 'red',
    'f': 'orange'
}

In [39]:
by_col = people.groupby(mapping, axis=1)

In [40]:
by_col.sum()

,blue,red
Joe,-0.052733,0.405282
Steve,2.177655,-1.345678
Wes,1.246634,-1.722699
Jim,0.371845,0.564355
Travis,-1.244359,0.506613


In [41]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [42]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


## Grouping with functions

Ok what??? This is black magic.

In [43]:
people.index

Index(['Joe', 'Steve', 'Wes', 'Jim', 'Travis'], dtype='object')

In [44]:
people.groupby(len).sum()

,a,b,c,d,e
3,-0.931680,-1.673741,1.013490,0.552255,1.852359
5,-0.688385,0.197563,0.994769,1.182886,-0.854856
6,0.144587,0.014133,-1.065756,-0.178602,0.347893


In [45]:
key_list = 'one one one two two'.split()
key_list

['one', 'one', 'one', 'two', 'two']

Mix and match:

In [46]:
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -1.346094 -1.215566  0.731149 -0.783882 -0.376605
  two -0.493079 -0.458175  0.282342  0.089503  1.515608
5 one -0.688385  0.197563  0.994769  1.182886 -0.854856
6 two  0.144587  0.014133 -1.065756 -0.178602  0.347893

## Groupping by index levels

In [47]:
cols = pd.MultiIndex.from_arrays(['US US US JP JP'.split(),
                                  [1, 3, 5, 1, 3]],
                                names=['city', 'tenor'])

In [48]:
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=cols)

hier_df

city         US                            JP          
tenor         1         3         5         1         3
0      1.085547  0.105668  0.242965  0.753570  0.705049
1      0.490140  1.722187  0.157368 -0.964909 -0.431739
2     -1.834230 -0.441567 -0.050613 -0.016294 -1.201187
3     -0.343074 -0.454909 -0.204524 -1.609929  0.201498

In [49]:
hier_df.groupby(level='city', axis=1).min()

city,JP,US
0,0.705049,0.105668
1,-0.964909,0.157368
2,-1.201187,-1.834230
3,-1.609929,-0.454909


Here we've created an index with two layers. We named one layer `city` and the other layer `tenor`. Those are the names we use to refer to those layers. The `groupby` statement shows how this is done.

## Data aggregation

In [50]:
# Quantile is available for Series objects, thus also available for groupby objects

df

,data1,data2,key1,key2
0,96.327036,96.766559,a,one
1,71.297520,92.673094,a,two
2,102.028623,101.530417,b,one
3,95.671300,117.166485,b,two
4,80.292342,83.297714,a,one


In [51]:
g = df.groupby('key1')

g['data1'].quantile(0.9)

key1
a     93.120097
b    101.392890
Name: data1, dtype: float64

### DIY aggregation with the `agg` method

Just write a function that aggregates arrays, then pass it to the grouped object's `agg` method.

In [52]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [53]:
g.agg(peak_to_peak)

,data1,data2
key1,,
a,25.029516,13.468845
b,6.357323,15.636068


### Other methods

In [54]:
g.describe()

data1                                                                    \
     count       mean        std       min        25%        50%         75%   
key1                                                                           
a      3.0  82.638966  12.678689  71.29752  75.794931  80.292342   88.309689   
b      2.0  98.849961   4.495306  95.67130  97.260631  98.849961  100.439292   

                 data2                                                 \
             max count        mean        std         min         25%   
key1                                                                    
a      96.327036   3.0   90.912456   6.904878   83.297714   87.985404   
b     102.028623   2.0  109.348451  11.056370  101.530417  105.439434   

                                          
             50%         75%         max  
key1                                      
a      92.673094   94.719827   96.766559  
b     109.348451  113.257468  117.166485

`describe` is not an aggregation function. But it still works.

## Column-wise and multiple function application

Here we use the `tips.csv` dataset provided by Wes on the GitHub for the book.

In [55]:
tips = pd.read_csv('data/tips.csv')

tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [56]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']

tips.head(6)

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [57]:
g = tips.groupby(['day', 'smoker'])

In [58]:
g_pct = g['tip_pct']

In [59]:
g_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

This is black magic. I swear it's too easy!! I'm not doing any work here!

In [60]:
g_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

But maybe you want different names for the columns?

In [61]:
# You can pass a tuple with ('name', 'func') elements

g_pct.agg([('Average', 'mean'), ('Std. Dev', 'std'), ('Range', peak_to_peak)])

Average  Std. Dev     Range
day  smoker                              
Fri  No      0.151650  0.028123  0.067349
     Yes     0.174783  0.051293  0.159925
Sat  No      0.158048  0.039767  0.235193
     Yes     0.147906  0.061375  0.290095
Sun  No      0.160113  0.042347  0.193226
     Yes     0.187250  0.154134  0.644685
Thur No      0.160298  0.038774  0.193350
     Yes     0.163863  0.039389  0.151240

In [62]:
funcs = 'count mean max'.split()
funcs

['count', 'mean', 'max']

In [63]:
result = g['tip_pct', 'total_bill'].agg(funcs)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

I swear that's just black magic. Really? All that as a one liner? That line is selecting just two columns from the original dataset. Then it is running three aggregation functions on each of them. And it gives you detail on day of the week and smoker/non-smoker?

Ok maybe that took three lines.

1. Group
1. List of functions
1. Aggregation

But still. Nice.

In [64]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]
ftuples

[('Durchschnitt', 'mean'),
 ('Abweichung', <function numpy.core.fromnumeric.var>)]

In [65]:
result = g['tip_pct', 'total_bill'].agg(ftuples)

result

tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [66]:
result['tip_pct']

Durchschnitt  Abweichung
day  smoker                          
Fri  No          0.151650    0.000791
     Yes         0.174783    0.002631
Sat  No          0.158048    0.001581
     Yes         0.147906    0.003767
Sun  No          0.160113    0.001793
     Yes         0.187250    0.023757
Thur No          0.160298    0.001503
     Yes         0.163863    0.001551

### What happens with a `dict`?

In [67]:
g.agg({
    'tip': np.max,
    'size': 'sum'
})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [68]:
g.agg({
    'tip_pct': 'min max mean std'.split(),
    'size': 'sum'
})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### Return data with non-hierarchical index

Sometimes the index doesn't need to be fancy.

In [69]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## Apply: General split-apply-combine

In [70]:
# Top five values by group

def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

In [71]:
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


### Top `n` rows by group using `apply`

In [72]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [73]:
# With args

tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

## Examples

### Describe by group

In [74]:
result = tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [75]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

### Suppressing the group keys

In [76]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


### Quantile and bucket analysis

In [77]:
frame = pd.DataFrame({
    'data1': np.random.randn(1000),
    'data2': np.random.randn(1000)
})
frame.head()

,data1,data2
0,-0.301837,-0.488842
1,-0.993221,-1.207806
2,-0.178678,-0.578099
3,0.387892,0.269224
4,2.749078,0.101558


In [78]:
quartiles = pd.cut(frame.data1, 4)
quartiles[:10]

0    (-1.66, -0.0954]
1    (-1.66, -0.0954]
2    (-1.66, -0.0954]
3    (-0.0954, 1.469]
4      (1.469, 3.034]
5    (-0.0954, 1.469]
6    (-1.66, -0.0954]
7    (-0.0954, 1.469]
8    (-1.66, -0.0954]
9      (1.469, 3.034]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.231, -1.66] < (-1.66, -0.0954] < (-0.0954, 1.469] < (1.469, 3.034]]

In [79]:
def get_stats(group):
    return {
        'min': group.min(),
        'max': group.max(),
        'count': group.count(),
        'mean': group.mean()
    }

In [80]:
g = frame.data2.groupby(quartiles)

In [81]:
g.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
"(-3.231, -1.66]",47.0,1.887960,0.100772,-2.007163
"(-1.66, -0.0954]",427.0,3.269427,0.013108,-2.499807
"(-0.0954, 1.469]",453.0,3.613020,-0.088934,-2.809270
"(1.469, 3.034]",73.0,2.592110,0.128860,-1.913320


Above are equal length buckets. Below are equal size buckets.

In [82]:
quantiles = pd.qcut(frame.data1, 10, labels=False)

In [83]:
g2 = frame.data2.groupby(quantiles)

In [84]:
g2.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
0,100.0,2.140218,0.027309,-2.499807
1,100.0,2.127724,-0.124396,-2.454742
2,100.0,3.097429,0.171503,-2.342693
3,100.0,3.269427,0.131163,-2.275489
4,100.0,2.272186,-0.102045,-2.309981
5,100.0,1.988723,-0.075402,-2.809270
6,100.0,2.144115,-0.226120,-2.213718
7,100.0,3.613020,0.019106,-1.859154
8,100.0,1.832686,-0.135688,-2.232038


### Fill missing values with group specific values

In [85]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1   -0.970867
2         NaN
3    0.159029
4         NaN
5   -0.285599
dtype: float64

In [86]:
s.fillna(s.mean())

0   -0.365812
1   -0.970867
2   -0.365812
3    0.159029
4   -0.365812
5   -0.285599
dtype: float64

In [87]:
states = 'Ohio NewYork Vermont Florida Oregon Nevada California Idaho'.split()
states[1] = 'New York'
states

['Ohio',
 'New York',
 'Vermont',
 'Florida',
 'Oregon',
 'Nevada',
 'California',
 'Idaho']

In [88]:
group_key = ['East'] * 4 + ['West'] * 4
group_key

['East', 'East', 'East', 'East', 'West', 'West', 'West', 'West']

In [89]:
data = pd.Series(np.random.randn(8), index=states)
data

Ohio          0.144958
New York     -0.716668
Vermont       0.510546
Florida       1.251124
Oregon        0.484708
Nevada       -1.154803
California    1.039882
Idaho         0.812801
dtype: float64

In [90]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio          0.144958
New York     -0.716668
Vermont            NaN
Florida       1.251124
Oregon        0.484708
Nevada             NaN
California    1.039882
Idaho              NaN
dtype: float64

In [91]:
data.groupby(group_key).mean()

East    0.226471
West    0.762295
dtype: float64

In [92]:
fill_mean = lambda g: g.fillna(g.mean())

In [93]:
data.groupby(group_key).apply(fill_mean)

Ohio          0.144958
New York     -0.716668
Vermont       0.226471
Florida       1.251124
Oregon        0.484708
Nevada        0.762295
California    1.039882
Idaho         0.762295
dtype: float64

And maybe we just have the fill value hard coded somewhere...

In [94]:
fill_values = {'East':0.5, 'West':-1}
fill_func = lambda g: g.fillna(fill_values[g.name])

In [95]:
data.groupby(group_key).apply(fill_func)

Ohio          0.144958
New York     -0.716668
Vermont       0.500000
Florida       1.251124
Oregon        0.484708
Nevada       -1.000000
California    1.039882
Idaho        -1.000000
dtype: float64

### Random sampling and permutation

A French deck with `pandas`. Aka, picking random cards.

In [96]:
suits = 'H S C D'.split()
card_val = (list(range(1,11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2,11)) + 'J Q K'.split()
cards = []
for suit in suits:
    cards.extend(str(num) + suit for num in base_names)
deck = pd.Series(card_val, index=cards)
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
QH     10
KH     10
dtype: int64

In [97]:
def draw(deck, n=5):
    return deck.sample(n)

draw(deck)

9C      9
7H      7
10C    10
4C      4
7S      7
dtype: int64

In [98]:
get_suit = lambda card: card[-1]

deck.groupby(get_suit).apply(draw, n=2)

C  2C      2
   10C    10
D  4D      4
   6D      6
H  KH     10
   5H      5
S  3S      3
   4S      4
dtype: int64

In [99]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

6C      6
AC      1
5D      5
3D      3
10H    10
QH     10
2S      2
10S    10
dtype: int64

## Group weighted average and correlation

In [100]:
df = pd.DataFrame({
    'category': 'a a a a b b b b'.split(),
    'data': np.random.randn(8),
    'weights': np.random.rand(8)
})
df

,category,data,weights
0,a,-0.091298,0.833799
1,a,1.658146,0.780315
2,a,0.257336,0.059318
3,a,-2.358375,0.802148
4,b,1.584588,0.091976
5,b,0.124910,0.209275
6,b,0.983411,0.346641
7,b,-0.013263,0.819331


In [101]:
g = df.groupby('category')

get_wavg = lambda g: np.average(g['data'], weights=g['weights'])

In [102]:
g.apply(get_wavg)

category
a   -0.266099
b    0.342080
dtype: float64

### Financial dataset example

In [103]:
close_px = pd.read_csv('data/stock_px_2.csv', parse_dates=True, index_col=0)

close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
AAPL    2214 non-null float64
MSFT    2214 non-null float64
XOM     2214 non-null float64
SPX     2214 non-null float64
dtypes: float64(4)
memory usage: 86.5 KB


In [104]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


Maybe we do a yearly correlation of daily returns?

In [105]:
rets = close_px.pct_change().dropna()

rets.head()

,AAPL,MSFT,XOM,SPX
2003-01-03,0.006757,0.001421,0.000684,-0.000484
2003-01-06,0.000000,0.017975,0.024624,0.022474
2003-01-07,-0.002685,0.019052,-0.033712,-0.006545
2003-01-08,-0.020188,-0.028272,-0.004145,-0.014086
2003-01-09,0.008242,0.029094,0.021159,0.019386


In [106]:
get_year = lambda x: x.year
by_year = rets.groupby(get_year)

by_year.size()

2003    251
2004    252
2005    252
2006    251
2007    251
2008    253
2009    252
2010    252
2011    199
dtype: int64

In [107]:
spx_corr = lambda x: x.corrwith(x['SPX'])
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [108]:
# or inter-column correlations

by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### Group-wise linear regression

The below code does not run. The issue is being worked on by the appropriate authorities.

In [109]:
import statsmodels.api as sm

ModuleNotFoundError: No module named 'statsmodels'

In [ ]:
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

In [ ]:
by_year.apply(regress, 'AAPL', ['SPX'])

I'll need to look into [statsmodels](http://www.statsmodels.org/dev/index.html).

The chapter ends with the `pivot_table` and `crosstab` methods. Those seem to be built on top of the `groupby` method and are there for convenience. They are tasks that happen often enough to warrant their own methods. I won't go into them now. I'll first get a good grasp of the `groupby` method and then look into those two.

## Rename a single column in one line

In [ ]:
# This needs to be fleshed out. Right now it is not valid code.

In [27]: df=df.rename(columns = {'two':'new_name'})

In [28]: df
Out[28]: 
  one three  new_name
0    1     a         9
1    2     b         8
2    3     c         7
3    4     d         6
4    5     e         5